In [4]:
import sys
import yfinance as yf
import requests
from bs4 import BeautifulSoup
from prophet import Prophet
from pandas.plotting import autocorrelation_plot
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta
import pandas_datareader as pdr
import pytictoc as tt
import plotly
from tickerTA import Ticker
from tickerTA import TechnicalAnalysis
from tradingStrategy1 import TradingStrategy1

# This is to avoid the SettingWithCopyWarning in pandas
pd.options.mode.copy_on_write = True 

In [5]:
def get_tickers500():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers500 = []
    for row in table.find_all('tr')[1:]:
        ticker = row.find('td').text.strip()
        tickers500.append(ticker)
    return tickers500

def add_z_score(data: pd.DataFrame, frame: str = 'Adj Close', window1 = 20) -> pd.DataFrame:
    data['Z Score ' + frame] = (data[frame] - data[frame].rolling(window= window1).mean()) / data[frame].rolling(window= window1).std()
    data.dropna(inplace=True)
    return data

def get_z_score(data: pd.DataFrame, frame: str = 'Adj Close', window1 = 20) -> pd.DataFrame:
    data['Z Score ' + frame] = (data[frame] - data[frame].rolling(window= window1).mean()) / data[frame].rolling(window= window1).std()
    data.dropna(inplace=True)
    return data['Z Score ' + frame]


def plot_total_cum_profit(self, start='2022-01-31', end='2024-01-31'):
    plt.figure(figsize=(20, 10))
    plt.plot(self['Buy Date'], self['Total Cum Profit %'], drawstyle="steps-post", label='Total Cumulative Profit', alpha=1)

    spy_df = yf.download('SPY', start, end, progress=False)
    spy_df['Control'] = spy_df['Close'] - spy_df['Open']
    spy_df['Control %'] = (spy_df['Control'] / spy_df['Open']) * 100
    spy_df['Control Cumulative %'] = spy_df['Control %'].cumsum()
    plt.plot(spy_df['Control Cumulative %'], drawstyle="steps-post", label='SPY', alpha=0.8)

    plt.title('Total Cumulative Profit % vs SPY')
    plt.legend(loc='upper left')
    plt.savefig('tradingStrategy1/total_cum_profit.png')
    # plt.show()

    plt.close()

In [6]:

if __name__ == '__main__':
    tickers500 = get_tickers500()
    stock_df = pd.DataFrame()
    tradingstrategy1_df = pd.DataFrame()

    for ticker in tickers500[0:2]:
        stock = Ticker(ticker, start='2022-01-31', end='2024-01-31')
        stock_df = stock_df._append(stock.df)
        techA = TechnicalAnalysis(stock)
        tradingS = TradingStrategy1(techA)
        tradingstrategy1_df = tradingstrategy1_df._append(tradingS.trades_df)

    # Recaulculate the cumulative profit % after reordering the rows
    tradingstrategy1_df['Total Cum Profit %'] = tradingstrategy1_df['Profit %'].cumsum()

    print(tradingstrategy1_df)
    plot_total_cum_profit(tradingstrategy1_df)



  Ticker   Buy Date  Buy Price  Buy P/E Ratio  Buy Fwd P/E Ratio  \
0    MMM 2022-04-19     146.85       0.000000           0.000000   
1    MMM 2022-06-13     136.73       0.000000           0.000000   
2    MMM 2022-08-29     128.79       0.000000           0.000000   
3    MMM 2022-12-16     121.31       0.000000           0.000000   
4    MMM 2023-05-17      98.42       0.000000           0.000000   
5    MMM 2023-09-26      94.36       0.000000           0.000000   
0    AOS 2022-04-01      64.62      23.349594          19.898384   
1    AOS 2022-08-29      57.91      23.349594          19.898384   
2    AOS 2022-12-16      56.23      23.349594          19.898384   
3    AOS 2023-06-01      64.17      23.349594          19.898384   
4    AOS 2023-08-21      68.87      23.349594          19.898384   

   Buy P/B Ratio    Buy RSI  Buy Upper BB  Buy Bollinger %b  Buy Lower BB  \
0       0.000000  59.093317    136.654199          0.632688    131.813801   
1       0.000000  15.713210  